# **Práctica: Flujos de Datos en Clasificación, Concept Drift, Agrupamiento, Anomalías, Texto y Ensembles**


## **Carga de Datos**

Cargamos el dataset que contiene los SMS con sus características extraídas y una columna que indica si es SPAM o no.

In [1]:
import pandas as pd

# Cargar los datos
df = pd.read_csv('spam_SMS_ampliado.csv')
df.columns

Index(['spam', 'text', 'num_caracteres', 'num_palabras', 'num_alfabeticos',
       'num_numericos', 'num_no_alfanum', 'num_divisas', 'num_mayusculas',
       'num_exclamaciones', 'num_interrogaciones', 'num_urls'],
      dtype='object')

In [2]:
df.head(5)

,spam,text,num_caracteres,num_palabras,num_alfabeticos,num_numericos,num_no_alfanum,num_divisas,num_mayusculas,num_exclamaciones,num_interrogaciones,num_urls
0,0,"Go until jurong point, crazy.. Available only ...",111,20,83,0,28,0,3,0,0,0
1,0,Ok lar... Joking wif u oni...,29,6,18,0,11,0,2,0,0,0
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155,28,97,25,33,0,10,0,0,0
3,0,U dun say so early hor... U c already then say...,49,11,33,0,16,0,2,0,0,0
4,0,"Nah I don't think he goes to usf, he lives aro...",61,13,47,0,14,0,2,0,0,0


In [3]:
len(df)

5574

## **Clasificación**

Vamos a entrenar y evaluar varios clasificadores online (en flujo de datos):
- Árbol Hoeffding (`HoeffdingTreeClassifier`).
- Árbol Adaptativo Hoeffding (`HoeffdingAdaptiveTreeClassifier`).
- Árbol de Decisión Extremadamente Rápido (`ExtremelyFastDecisionTreeClassifier`).

Estos tres modelos utilizan árboles de decisión para realizar la clasificación junto con el teorema de Hoeffding, el primero lo hemos elegido porque debería dar buenos resultados al estar tratando con un flujo de datos estable, sin embargo, queremos ver que efecto tiene el AdaptiveHoeffdingTree ya que este modelo tiene la capacidad de adaptarse al concept drift, por la estructura de nuestros datos, creemos que no hay mucho concept drift, sin embargo, pensamos que siempre puede haber un poco por lo que creemos que aplicando el árbol adaptativo podremos mejorar los resultados respecto al estándar, ya que, en caso de que haya una ligera desviación en las características de los datos, el modelo pueda detectarlo y adaptarse a ello.
Por último hemos querido probar el árbol extremadamente rápido, para ver como afecta la optimización de este modelo frente al modelo estable en los resultados, teniendo en mente que el resultado podría empeorar.
 
Evaluaremos cada uno utilizando la métrica F1.

In [4]:
from river import tree, metrics

model_standard = tree.HoeffdingTreeClassifier()

metric = metrics.F1()

for index, row in df.iterrows():
    x = {
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

    y = row['spam'] 
    y_pred = model_standard.predict_one(x)
    model_standard.learn_one(x, y)
    metric.update(y, y_pred)
    
print(f'F1 Score para HoeffdingTreeClassifier: {metric}')

F1 Score para HoeffdingTreeClassifier: F1: 86.65%


In [5]:
model_adaptive = tree.HoeffdingAdaptiveTreeClassifier()

metric = metrics.F1()

for index, row in df.iterrows():
    x = {
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

    y = row['spam'] 
    y_pred = model_adaptive.predict_one(x)
    model_adaptive.learn_one(x, y)
    metric.update(y, y_pred)
    
print(f'F1 Score para HoeffdingAdaptiveTreeClassifier: {metric}')

F1 Score para HoeffdingAdaptiveTreeClassifier: F1: 86.85%


In [6]:
model_extreme = tree.ExtremelyFastDecisionTreeClassifier(grace_period=120)

metric = metrics.F1()

for index, row in df.iterrows():
    x = {
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

    y = row['spam'] 
    y_pred = model_extreme.predict_one(x)
    model_extreme.learn_one(x, y)
    metric.update(y, y_pred)

print(f'F1 Score para ExtremelyFastDecisionTreeClassifier: {metric}')

F1 Score para ExtremelyFastDecisionTreeClassifier: F1: 86.16%


Como era de esperar la precisión medida por el F1-Score es menor en el árbol extremadamente rápido, aunque esta disminución en el resultado es muy ligera, lo que nos lleva a pensar que en una implantación real en la que nuestra detección de Spam en SMSs tiene que ser lo más rápido posible debido a la forma en la que entran los datos, lo consideramos una buena opción para implementaciones en tiempo real.
Además, hemos visto como el árbol adaptativo es efectivamente, más preciso que el estándar, gracias a su capacidad de adaptarse al concept drift.

## **Detección de Concept Drift**

Tras clasificar correctamente los datos y ver cómo el árbol adaptativo se ha adaptado al concept drift (ya que ha mejorado sus predicciones frente a las del árbol estándar), pasamos a la detección de este concept drift, para ello utilizaremos los siguientes detectores, tanto de data drift como de model drift:

- **KSWIN (`KSWIN`)**: Detector de cambios basado en ventanas deslizantes.
- **EDDM (`EDDM`)**: Monitor de desviación por error esperado.
- **DDM (`DDM`)**: Monitor de desviación por error de detección.

Vamos a implementar estos detectores sobre los tres modelos iniciales:
1. HoeffdingTreeClassifier
2. HoeffdingAdaptiveTreeClassifier
3. ExtremelyFastDecisionTreeClassifier

La idea de probarlos en los tres modelos es sobre todo para comparar el número de detecciones que ocurren entre el modelo estándar y el extremadamente rápido frente al adaptativo, que, según creemos debería detectar menos cambios ya que se va adaptando a ellos durante la clasificación.

### **Concept Drift en HoeffdingTreeClassifier**

Entrenaremos un modelo estándar de árbol Hoeffding y usaremos los detectores de concept drift para evaluar si se detectan cambios en los datos durante el flujo.

In [8]:
from river import tree, metrics, drift
import river
print(river.__version__)

model_standard = tree.HoeffdingTreeClassifier()

metric = metrics.F1()

kswin = drift.KSWIN()
eddm = drift.binary.EDDM()
ddm = drift.binary.DDM()

for index, row in df.iterrows():
    x = {
        'num_caracteres': row['num_caracteres'],
        'num_palabras': row['num_palabras'],
        'num_alfabeticos': row['num_alfabeticos'],
        'num_numericos': row['num_numericos'],
        'num_no_alfanum': row['num_no_alfanum'],
        'num_divisas': row['num_divisas'],
        'num_mayusculas': row['num_mayusculas'],
        'num_exclamaciones': row['num_exclamaciones'],
        'num_interrogaciones': row['num_interrogaciones'],
        'num_urls': row['num_urls']
    }

    y = row['spam'] 
    y_pred = model_standard.predict_one(x)
    model_standard.learn_one(x, y)
    metric.update(y, y_pred)
    
    eddm.update(int(y == y_pred))
    ddm.update(int(y == y_pred))
    kswin.update(int((x['num_mayusculas'])))
    
    if eddm.drift_detected: 
        print(f"EDDM - Model drift detectado en el ejemplo {index} ({metric})")
    if ddm.drift_detected:
        print(f"DDM - Model drift detectado en el ejemplo {index} ({metric})")
    if kswin.drift_detected:
        print(f"KSWIN - Data drift detectado en el ejemplo {index} ({metric})")

print(f'Resultado final (HoeffdingTreeClassifier): {metric}')

0.21.2
EDDM - Model drift detectado en el ejemplo 48 (F1: 14.29%)
EDDM - Model drift detectado en el ejemplo 111 (F1: 16.00%)
EDDM - Model drift detectado en el ejemplo 174 (F1: 53.57%)
EDDM - Model drift detectado en el ejemplo 224 (F1: 52.46%)
EDDM - Model drift detectado en el ejemplo 297 (F1: 59.26%)
EDDM - Model drift detectado en el ejemplo 389 (F1: 68.47%)
DDM - Model drift detectado en el ejemplo 440 (F1: 71.43%)
EDDM - Model drift detectado en el ejemplo 478 (F1: 73.13%)
EDDM - Model drift detectado en el ejemplo 557 (F1: 75.64%)
KSWIN - Data drift detectado en el ejemplo 2469 (F1: 86.13%)
Resultado final (HoeffdingTreeClassifier): F1: 86.65%


### **Concept Drift en HoeffdingAdaptiveTreeClassifier**

Ahora, vamos a implementar el árbol adaptativo Hoeffding, que puede adaptarse a los cambios detectados en el concepto de los datos, y lo evaluamos con los detectores de drift.

In [9]:
# crear, entrenar y evaluar modelo
model_adaptive = tree.HoeffdingAdaptiveTreeClassifier()

metric = metrics.F1()

# detectores de drift
kswin = drift.KSWIN()
eddm = drift.binary.EDDM()
ddm = drift.binary.DDM()

for index, row in df.iterrows():
    x = {
        'num_caracteres': row['num_caracteres'],
        'num_palabras': row['num_palabras'],
        'num_alfabeticos': row['num_alfabeticos'],
        'num_numericos': row['num_numericos'],
        'num_no_alfanum': row['num_no_alfanum'],
        'num_divisas': row['num_divisas'],
        'num_mayusculas': row['num_mayusculas'],
        'num_exclamaciones': row['num_exclamaciones'],
        'num_interrogaciones': row['num_interrogaciones'],
        'num_urls': row['num_urls']
    }

    y = row['spam'] 
    y_pred = model_adaptive.predict_one(x)
    model_adaptive.learn_one(x, y)
    metric.update(y, y_pred)
    
    eddm.update(int(y == y_pred))
    ddm.update(int(y == y_pred))
    kswin.update(int((x['num_mayusculas'])))
    
    if eddm.drift_detected: 
        print(f"EDDM - Model drift detectado en el ejemplo {index} ({metric})")
    if ddm.drift_detected:
        print(f"DDM - Model drift detectado en el ejemplo {index} ({metric})")
    if kswin.drift_detected:
        print(f"KSWIN - Data drift detectado en el ejemplo {index} ({metric})")

print(f'Resultado final (HoeffdingAdaptiveTreeClassifier): {metric}')

EDDM - Model drift detectado en el ejemplo 51 (F1: 13.33%)
EDDM - Model drift detectado en el ejemplo 114 (F1: 33.33%)
EDDM - Model drift detectado en el ejemplo 161 (F1: 53.85%)
EDDM - Model drift detectado en el ejemplo 216 (F1: 56.67%)
DDM - Model drift detectado en el ejemplo 216 (F1: 56.67%)
EDDM - Model drift detectado en el ejemplo 304 (F1: 62.50%)
EDDM - Model drift detectado en el ejemplo 378 (F1: 69.81%)
KSWIN - Data drift detectado en el ejemplo 2044 (F1: 86.88%)
KSWIN - Data drift detectado en el ejemplo 3371 (F1: 86.44%)
KSWIN - Data drift detectado en el ejemplo 3496 (F1: 86.50%)
KSWIN - Data drift detectado en el ejemplo 4622 (F1: 86.97%)
Resultado final (HoeffdingAdaptiveTreeClassifier): F1: 87.33%


### **Concept Drift en ExtremelyFastDecisionTreeClassifier**

Finalmente, probamos el árbol de decisiones extremadamente rápido, junto con los detectores de drift.


In [10]:
# crear, entrenar y evaluar modelo
model_extreme = tree.ExtremelyFastDecisionTreeClassifier(grace_period=120)
metric = metrics.F1()

# detectores de drift
kswin = drift.KSWIN()
eddm = drift.binary.EDDM()
ddm = drift.binary.DDM()

for index, row in df.iterrows():
    x = {
        'num_caracteres': row['num_caracteres'],
        'num_palabras': row['num_palabras'],
        'num_alfabeticos': row['num_alfabeticos'],
        'num_numericos': row['num_numericos'],
        'num_no_alfanum': row['num_no_alfanum'],
        'num_divisas': row['num_divisas'],
        'num_mayusculas': row['num_mayusculas'],
        'num_exclamaciones': row['num_exclamaciones'],
        'num_interrogaciones': row['num_interrogaciones'],
        'num_urls': row['num_urls']
    }

    y = row['spam'] 
    y_pred = model_extreme.predict_one(x)
    model_extreme.learn_one(x, y)
    metric.update(y, y_pred)
    
    eddm.update(int(y == y_pred))
    ddm.update(int(y == y_pred))
    kswin.update(int((x['num_mayusculas'])))
    
    if eddm.drift_detected: 
        print(f"EDDM - Model drift detectado en el ejemplo {index} ({metric})")
    if ddm.drift_detected:
        print(f"DDM - Model drift detectado en el ejemplo {index} ({metric})")
    if kswin.drift_detected:
        print(f"KSWIN - Data drift detectado en el ejemplo {index} ({metric})")

print(f'Resultado final (ExtremelyFastDecisionTreeClassifier): {metric}')

EDDM - Model drift detectado en el ejemplo 48 (F1: 14.29%)
EDDM - Model drift detectado en el ejemplo 111 (F1: 16.00%)
EDDM - Model drift detectado en el ejemplo 161 (F1: 50.00%)
EDDM - Model drift detectado en el ejemplo 216 (F1: 53.57%)
DDM - Model drift detectado en el ejemplo 414 (F1: 69.64%)
EDDM - Model drift detectado en el ejemplo 654 (F1: 79.14%)
EDDM - Model drift detectado en el ejemplo 845 (F1: 82.17%)
EDDM - Model drift detectado en el ejemplo 914 (F1: 83.10%)
KSWIN - Data drift detectado en el ejemplo 2465 (F1: 84.45%)
KSWIN - Data drift detectado en el ejemplo 3354 (F1: 85.12%)
KSWIN - Data drift detectado en el ejemplo 3494 (F1: 85.20%)
KSWIN - Data drift detectado en el ejemplo 4619 (F1: 85.11%)
Resultado final (ExtremelyFastDecisionTreeClassifier): F1: 86.16%


## **Agrupamiento y Detección de Anomalías**

En esta sección, utilizamos el método no supervisado de KMeans para agrupar los datos en clústeres. Posteriormente, normalizamos los datos y detectamos las anomalías en función de la distancia euclidiana de cada dato al centroide de su clúster asignado.

### Pasos:
1. Normalizar los datos usando `StandardScaler`.
2. Agrupar los datos usando `KMeans` con 5 clústeres.
3. Calcular la distancia al centroide del clúster asignado para cada dato.
4. Detectar anomalías si la distancia al centroide es mayor a un umbral definido.

In [11]:
from river import preprocessing, cluster
import math

scaler = preprocessing.StandardScaler()

kmeans = cluster.KMeans(n_clusters=5, seed=10)

threshold = 10.0

anomalies_detected = 0

def euclidean_distance(point, center):
    return math.sqrt(sum((point[feature] - center[feature]) ** 2 for feature in point))


### **Flujo de Datos para Detección de Anomalías**

En esta sección, simulamos el procesamiento en flujo continuo de datos (streaming) y escalamos las características y las agrupamos en clústeres utilizando KMeans. Detectaremos anomalías cuando la distancia al centroide sea mayor que el umbral definido.

In [12]:
for index, row in df.iterrows():
    features = {
        'num_caracteres': float(row['num_caracteres']),
        'num_palabras': float(row['num_palabras']),
        'num_alfabeticos': float(row['num_alfabeticos']),
        'num_numericos': float(row['num_numericos']),
        'num_no_alfanum': float(row['num_no_alfanum']),
        'num_divisas': float(row['num_divisas']),
        'num_mayusculas': float(row['num_mayusculas']),
        'num_exclamaciones': float(row['num_exclamaciones']),
        'num_interrogaciones': float(row['num_interrogaciones']),
        'num_urls': float(row['num_urls'])
    }

    scaler.learn_one(features)
    
    features_scaled = scaler.transform_one(features)

    cluster_id = kmeans.predict_one(features_scaled)

    centroids = kmeans.centers

    if centroids:
        centroid = centroids[cluster_id]

        distance_to_centroid = euclidean_distance(features_scaled, centroid)

        if distance_to_centroid > threshold:
            anomalies_detected += 1
            print(f"Anomalía detectada en {index} con distancia {distance_to_centroid:.2f} ")

    kmeans.learn_one(features_scaled)

print(f"Total de anomalías detectadas: {anomalies_detected}")

Anomalía detectada en 445 con distancia 10.19 
Anomalía detectada en 793 con distancia 13.29 
Anomalía detectada en 1085 con distancia 25.62 
Anomalía detectada en 1579 con distancia 10.47 
Anomalía detectada en 2158 con distancia 11.74 
Anomalía detectada en 2503 con distancia 19.65 
Anomalía detectada en 2676 con distancia 14.59 
Anomalía detectada en 2791 con distancia 10.40 
Anomalía detectada en 2849 con distancia 10.41 
Anomalía detectada en 3564 con distancia 10.27 
Anomalía detectada en 5019 con distancia 10.54 
Anomalía detectada en 5083 con distancia 10.15 
Anomalía detectada en 5106 con distancia 11.96 
Anomalía detectada en 5266 con distancia 11.11 
Anomalía detectada en 5542 con distancia 10.11 
Total de anomalías detectadas: 15


## **Tratamiento de Texto**

En esta sección, compararemos dos técnicas para la extracción de características de texto:
- **Bag of Words (BOW)**: Simplemente cuenta las ocurrencias de cada palabra en el texto.
- **TF-IDF (Term Frequency-Inverse Document Frequency)**: Pondera la frecuencia de las palabras considerando también su relevancia en el conjunto total de datos.

Ambas técnicas serán combinadas con un clasificador Naive Bayes para evaluar cuál de las dos consigue mejores resultados en la clasificación de mensajes SPAM.

### **Bag of Words con Naive Bayes**

En este apartado implementamos la técnica de Bag of Words junto con el clasificador Naive Bayes. Entrenaremos el modelo y evaluaremos su rendimiento usando la métrica **F1 Score**.

In [13]:
from river import feature_extraction, naive_bayes, compose, metrics

bow = feature_extraction.BagOfWords(lowercase=True)

model_bayes = naive_bayes.MultinomialNB()

pipeline_BowNb = compose.Pipeline(('vectorizer', bow), ('nb', model_bayes))

metric = metrics.F1()

for index, row in df.iterrows():
    text = row['text']
    y = row['spam']
    
    y_pred = pipeline_BowNb.predict_one(text)
    
    metric.update(y, y_pred)
    
    pipeline_BowNb.learn_one(text, y)

print(f"F1 Score final con Bag of Words: {metric.get()}")


F1 Score final con Bag of Words: 0.9002590673575129


### **TF-IDF con Naive Bayes**

En esta parte, utilizamos la técnica de TF-IDF junto con el clasificador Naive Bayes. Evaluaremos su rendimiento en flujo de datos de la misma manera que lo hicimos con Bag of Words.

In [14]:
tfidf = feature_extraction.TFIDF(lowercase=True, ngram_range=(1,1))
model_bayes = naive_bayes.MultinomialNB()
pipeline_TFIDFNb = compose.Pipeline(('vectorizer', tfidf), ('TFI-IDF', model_bayes))

metric = metrics.F1()

for index, row in df.iterrows():
    text = row['text']
    y = row['spam']
    
    y_pred = pipeline_TFIDFNb.predict_one(text)
    
    metric.update(y, y_pred)
    
    pipeline_TFIDFNb.learn_one(text, y)

print(f"F1 Score final con TF-IDF: {metric.get()}")

F1 Score final con TF-IDF: 0.7392405063291139


## **Ensembles**
Aplicamos el modelo AdaBoostingClassifier de la librería RIver sobre el clasificador que mejor resultado nos dio al inicio, el HOeffdingAdaptiveTreeClassifier.



In [26]:
from river import metrics, tree, ensemble

metric = metrics.F1()
model_adaptive = tree.HoeffdingAdaptiveTreeClassifier()
model_ensemble_AdaBoost = ensemble.AdaBoostClassifier(model=model_adaptive, n_models=5)

for index, row in df.iterrows():
    x = {
        'num_caracteres': row['num_caracteres'],
        'num_palabras': row['num_palabras'],
        'num_alfabeticos': row['num_alfabeticos'],
        'num_numericos': row['num_numericos'],
        'num_no_alfanum': row['num_no_alfanum'],
        'num_divisas': row['num_divisas'],
        'num_mayusculas': row['num_mayusculas'],
        'num_exclamaciones': row['num_exclamaciones'],
        'num_interrogaciones': row['num_interrogaciones'],
        'num_urls': row['num_urls']
    }

    y = row['spam']
    y_pred = model_ensemble_AdaBoost.predict_one(x)
    model_ensemble_AdaBoost.learn_one(x, y)
    metric.update(y, y_pred)

print(f"F1 Score final: {metric}")



F1 Score final: F1: 87.96%


Aplicamos el modelo ADWINBoostingClassifier de la librería RIver sobre el clasificador que mejor resultado nos dio al inicio, el HOeffdingAdaptiveTreeClassifier.

In [28]:
from river import metrics, tree, ensemble

metric = metrics.F1()
model_adaptive = tree.HoeffdingAdaptiveTreeClassifier()
model_ensemble_AdWINBoosting = ensemble.ADWINBoostingClassifier(model=model_adaptive, n_models=5)

for index, row in df.iterrows():
    x = {
        'num_caracteres': row['num_caracteres'],
        'num_palabras': row['num_palabras'],
        'num_alfabeticos': row['num_alfabeticos'],
        'num_numericos': row['num_numericos'],
        'num_no_alfanum': row['num_no_alfanum'],
        'num_divisas': row['num_divisas'],
        'num_mayusculas': row['num_mayusculas'],
        'num_exclamaciones': row['num_exclamaciones'],
        'num_interrogaciones': row['num_interrogaciones'],
        'num_urls': row['num_urls']
    }

    y = row['spam']
    y_pred = model_ensemble_AdWINBoosting.predict_one(x)
    model_ensemble_AdWINBoosting.learn_one(x, y)
    metric.update(y, y_pred)

print(f"F1 Score final: {metric}")


F1 Score final: F1: 87.48%


## **Flujo de datos Simulado**

In [16]:
import time

try:
    for index, row in df.iterrows():
        X = {
            'text': row['text'],
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

        y = row['spam'] 
        
        print(f"Datos recibidos: {X}, Etiqueta: {y}")
        
        time.sleep(0.5)
        
except KeyboardInterrupt:
    print('Proceso detenido por el usuario.')

Datos recibidos: {'text': 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'num_caracteres': 111, 'num_palabras': 20, 'num_alfabeticos': 83, 'num_numericos': 0, 'num_no_alfanum': 28, 'num_divisas': 0, 'num_mayusculas': 3, 'num_exclamaciones': 0, 'num_interrogaciones': 0, 'num_urls': 0}, Etiqueta: 0
Datos recibidos: {'text': 'Ok lar... Joking wif u oni...', 'num_caracteres': 29, 'num_palabras': 6, 'num_alfabeticos': 18, 'num_numericos': 0, 'num_no_alfanum': 11, 'num_divisas': 0, 'num_mayusculas': 2, 'num_exclamaciones': 0, 'num_interrogaciones': 0, 'num_urls': 0}, Etiqueta: 0
Datos recibidos: {'text': "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'num_caracteres': 155, 'num_palabras': 28, 'num_alfabeticos': 97, 'num_numericos': 25, 'num_no_alfanum': 33, 'num_divisas': 0, 'num_mayusculas': 10, 'num_exclamaciones